This a notebook for COMP5349 Cloud Computing Assignment 2, implementing Spark Framework for data analyiss of online review data set released by Amazon on US sites. It provide various analytic phases to evaluate the data and calculate summary statistics, removing unwanted and meanning less data and finally performing some similarity analysis between review sentences.

The original data include product review information of Video DVD products from Amazon between 1995 and 2015. Each row reprenst one review given by one custoer on one prodcut and containing 15 columns of informaiton. This work load has only applied anaylisis based 5 columns of informaiton: customer_id, review_id, product_id, star_rating, review_body, while the additional information are striped out.

This notebook is designed to run on AWS EMR and obtain data from S3, after it's been downloaed into the bucket.

The first cell contains all necessary import and initinalization, the second cell create a SparkSession, the entry point for Spark SQL Click run cell button on the menu to run the first two cells. ???

In [1]:
from pyspark.sql import SparkSession,dataframe
from pyspark.sql.types import StructType,StructField,StringType,IntegerType,FloatType
from nltk.tokenize import sent_tokenize
import pyspark.sql.functions as fc
import numpy as np 
import tensorflow as tf
import tensorflow_hub as hub 
import scipy as sp 
import sklearn


VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
5,application_1558344667099_0006,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


In [5]:
#start spark session for program entry  
spark = SparkSession.builder \
.master("local[3]") \
.appName("Comp5349 Assignment 2") \
.getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

VBox()

In [6]:
data = "s3://comp5349-wdai9162/amazon_reviews_us_Video_DVD_v1_00.tsv.gz"
schema = StructType([
    StructField("marketplace", StringType(), True),
    StructField("customer_id", StringType(), True),
    StructField("review_id", StringType(), True),
    StructField("product_id", StringType(), True),
    StructField("product_parent", IntegerType(), True),
    StructField("product_title",StringType(),True),
    StructField("product_category",StringType(),True),
    StructField("star_rating",IntegerType(),True),
    StructField("helpful_votes",IntegerType(),True),
    StructField("total_votes",IntegerType(),True),
    StructField("vine",StringType(),True),
    StructField("verified_purchase",StringType(),True),
    StructField("review_headline",StringType(),True),
    StructField("review_body",StringType(),True),
    StructField("review_date",StringType(),True)])


VBox()

In [7]:
#read the data according to the defined schema
reviews = spark.read.csv(data,sep='\t',header=True, schema=schema)

#remove unrelated data column, keeping only customer_id, review_id, product_id, star_rating, review_body
#cache it to the default storage level 
clean_reviews = reviews.drop("marketplace") \
.drop("product_parent") \
.drop("product_title") \
.drop("product_category") \
.drop("helpful_votes") \
.drop("total_votes") \
.drop("vine") \
.drop("verified_purchase") \
.drop("review_headline") \
.drop("review_date") \

#show the first 20 records, job 1 - 236.399s ,237.564s
#clean_reviews.show(20)

VBox()

In [ ]:
#Stage 1.1 

#total number of reviews - approx 66.0s
total = clean_reviews.count()
print ("the total number of reviews is %d" %(total))

#the number of unique users - approx 84.0s 
uniq_usr = clean_reviews.dropDuplicates(["customer_id"]).count()
print ("the number of unique users is %d" %(uniq_usr))

#the number of unique products - approx 72.1s 
uniq_prdct = clean_reviews.dropDuplicates(["product_id"]).count()
print ("the number of unique products is %d" %(uniq_prdct))

In [ ]:
#Stage 1.2 

#the largest number of reviews published by a single user - approx 106.0s 
max_reviews_byUsr = clean_reviews.groupBy("customer_id").count().agg({"count": "max"}).collect()
print ("the largest number of reviews published by a single user is %d" %(max_reviews_byUsr[0]))

#the top 10 users ranked by the number of reviews they publish - approx 83.0s 
sorted_list_user = clean_reviews.groupBy("customer_id").count().orderBy("count",ascending=[0]).show(10)
 
#the median number of reviews published by a user - approx 85.0s 
def calc_median(column):
    try:
        median = np.median(column) 
        return round(float(median),2)
    except Exception:
        return "input error" #return nothing if given value is wrong 

median_finder = fc.udf(calc_median,FloatType()) #define function for spark datafram 

review_list = clean_reviews.groupBy("customer_id").count().agg(fc.collect_list("count").alias("counts"))
review_list = review_list.withColumn("median",median_finder("counts")).show()

In [ ]:
#Stage 1.3 

#the largest number of reviews written for a single product - approx 105.0s 
max_reviews_Product = clean_reviews.groupBy("product_id").count().agg({"count": "max"}).collect()
print ("the largest number of reviews written for a single product %d" %(max_reviews_Product[0]))
max_review = max_reviews_Product[0]

#the top 10 products ranked by the number of reviews they have - approx 84.0s 
sorted_products = clean_reviews.groupBy("product_id").count().orderBy("count",ascending=[0])
sorted_products.show(10)

#the median number of reviews a product has
product_list = clean_reviews.groupBy("product_id").count().agg(fc.collect_list("count").alias("counts"))
product_list = product_list.withColumn("median",median_finder("counts")).show()


In [20]:
#Stage 2.1 

median_row=[]
def getmedian (median):
    listnum=[median[i] for i in range(len(median))]
    listnum.sort()
    lnum=len(median)
    if(lnum%2==1):
        i=int((lnum+1)/2)-1
        return listnum[i]
    else:
        i=int(lnum/2)-1
        return (listnum[i] + listnum[i + 1]) / 2
    
# reviews with less than two sentences in the review_body - approx 1241s 
clean_reviews_filter = clean_reviews.dropna('all',None,['review_body']).rdd.filter(lambda row:(len(sent_tokenize(row.review_body)))>=2)
clean_reviews_filter.count()


VBox()

4016567

In [ ]:
#reviews published by users with less than median number of reviews published approx - 1479.639s 
f=clean_reviews_filter.groupBy(lambda row:(row.customer_id)).countByKey()

#clean_reviews_filter_2 ===== FINAL RESULT 
clean_reviews_filter_2=clean_reviews_filter.map(lambda row:(row.customer_id,f[row.customer_id]))


new_list=[]
for i in range(clean_reviews_filter_2.count()):
    new_list.append(clean_reviews_filter_2.collect()[i][1])
    
num_median_reviewC=getmedian(new_list)

clean_reviews_filter_2.filter(lambda row :row[1]>num_median_review)
new_list1=[]
for i in range(clean_reviews_filter_2.count()):
    new_list1.append(clean_reviews_filter_2.collect()[i][0])
clean_reviews_filter_2.filter(lambda row:row.customer_id!=[new_list1[i] for i in len(new_list1)])

# new_rating2.take(10)

VBox()

In [ ]:
# reviews from products with less than median number of reviews received
g=clean_reviews_filter_2.groupBy(lambda row:row.product_id).countByKey()
new_rating4=clean_reviews_filter_2.map(lambda row:(row.product_id,g[row.product_id]))
# new_rating4.collect()[0][1]
new_list2=[]
for i in range(new_rating4.count()):
    new_list2.append(new_rating4.collect()[i][1])

num_median_reviewP=getmedian(new_list2)

new_rating4.filter(lambda row:row[1]>num_median_reviewP)

new_list3=[]
for i in range(new_rating4.count()):
    new_list3.append(new_rating4.collect()[i][0])

clean_reviews_filter_2.filter(lambda row: row.product_id!=[new_list3[i] for i in len(new_list3)])


In [ ]:
#Stage 3: Similarity analysis with Sentence Embedding

#5.1 Positive vs. Negative Reviews

#pick the 10th most popular prodcut by review number according to result calculated from stage 1.3 - approx 95.0s 
pop_product_10th = sorted_products.select("product_id").collect()[9][0]
stage3_data = clean_reviews.filter(clean_reviews.product_id == pop_product_10th) 

positive_class = stage3_data.filter(stage3_data.star_rating >= 4)
positive_review = positive_class.select("review_body").rdd.flatMap(lambda x: x).collect()

negative_class = stage3_data.filter(stage3_data.star_rating <= 2)
negative_review = negative_class.select("review_body").rdd.flatMap(lambda x: x).collect()

In [ ]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" 
embed = hub.Module(module_url) 

#sentence segamentation for both positive lists 
positive_review_sent = [] 

for i in range (len(positive_review)):
    positive_review_sent.append(sent_tokenize(positive_review[i]))
    
f_positive_sent = [y for x in positive_review_sent for y in x] #flat the list 

#sentence segamentation for both negative lists 
negative_review_list = [] 
for i in range (len(negative_review)):
    negative_review_list.append(sent_tokenize(negative_review[i]))

f_negative_sent = [y for x in negative_review_list for y in x] #flat the list 

In [ ]:
print (len(positive_review))
print (len(positive_review_sent ))
print (len(f_positive_sent))
print (f_positive_sent)

#filter "!","?", clean data 

In [ ]:
#Use Google Pre-trained universal sentence encoder to encode each sentence to 512 dimension vector 
with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    sentence_embeddings_pos = session.run(embed(f_positive_sent))
    sentence_embeddings_neg = session.run(embed(f_negative_sent))

In [ ]:
print (sentence_embeddings_pos.shape)
print (sentence_embeddings_neg.shape)

In [ ]:
#5.2 Intra-Class Similarity, 0 = similar, 2 = different/opposite 
from scipy.spatial.distance import pdist

#calculate pairwise distances between observations in n-dimensional space.
dist_matrix_positive = pdist(sentence_embeddings_pos,"cosine")
dist_matrix_negative = pdist(sentence_embeddings_neg,"cosine")

average_dist_positive = np.mean(dist_matrix_positive)
average_dist_negative = np.mean(dist_matrix_negative)

print (average_dist_positive)
print (average_dist_negative)

In [ ]:
#calculate class center 
